In [0]:
import pyspark.sql.functions as fn
from pyspark.sql.avro.functions import from_avro
from confluent_kafka.schema_registry import SchemaRegistryClient
import ssl
import pyspark.sql.functions as fn
from pyspark.sql.types import StringType
from pyspark.sql.functions import col


In [0]:
confluentClusterName = dbutils.secrets.get("fhir", "confluentClusterName")
confluentBootstrapServers = dbutils.secrets.get("fhir", "confluentBootstrapServers")
confluentApiKey = dbutils.secrets.get("fhir", "confluentApiKey")
confluentSecret = dbutils.secrets.get("fhir", "confluentSecret")
schemaRegistryUrl = dbutils.secrets.get("fhir", "schemaRegistryUrl")
confluentRegistryApiKey = dbutils.secrets.get("fhir", "confluentRegistryApiKey")
confluentRegistrySecret = dbutils.secrets.get("fhir", "confluentRegistrySecret")
confluentTopicName = "patients"
deltaTablePath = dbutils.secrets.get("fhir", "deltaTablePath") + confluentTopicName
checkpointPath = dbutils.secrets.get("fhir", "checkpointPath") + confluentTopicName

In [0]:
schema_registry_conf = {
    'url': schemaRegistryUrl,
    'basic.auth.user.info': '{}:{}'.format(confluentRegistryApiKey, confluentRegistrySecret)}

schema_registry_client = SchemaRegistryClient(schema_registry_conf)

In [0]:
binary_to_string = fn.udf(lambda x: str(int.from_bytes(x, byteorder='big')), StringType())
streamTestDf = (
  spark
  .readStream
  .format("kafka")
  .option("kafka.bootstrap.servers", confluentBootstrapServers)
  .option("kafka.security.protocol", "SASL_SSL")
  .option("kafka.sasl.jaas.config", "kafkashaded.org.apache.kafka.common.security.plain.PlainLoginModule required username='{}' password='{}';".format(confluentApiKey, confluentSecret))
  .option("kafka.ssl.endpoint.identification.algorithm", "https")
  .option("kafka.sasl.mechanism", "PLAIN")
  .option("subscribe", confluentTopicName)
  .option("startingOffsets", "latest")
  .option("failOnDataLoss", "false")
  .load()
  .withColumn('key', fn.col("key").cast(StringType()))
  .withColumn('fixedValue', fn.expr("substring(value, 6, length(value)-5)"))
  .withColumn('valueSchemaId', binary_to_string(fn.expr("substring(value, 2, 4)")))
  .select('topic', 'partition', 'offset', 'timestamp', 'timestampType', 'key', 'valueSchemaId','fixedValue')
)

In [0]:
# display(streamTestDf)

In [0]:
# def parseAvroDataWithSchemaId(df, ephoch_id):
#   cachedDf = df.cache()
#   fromAvroOptions = {"mode":"FAILFAST"}
#   def getSchema(id):
#     return str(schema_registry_client.get_schema(id).schema_str)
#   distinctValueSchemaIdDF = cachedDf.select(fn.col('valueSchemaId').cast('integer')).distinct()
#   for valueRow in distinctValueSchemaIdDF.collect():
#     currentValueSchemaId = sc.broadcast(valueRow.valueSchemaId)
#     currentValueSchema = sc.broadcast(getSchema(currentValueSchemaId.value))
#     filterValueDF = cachedDf.filter(fn.col('valueSchemaId') == currentValueSchemaId.value)
#     filterValueDF \
#       .select('topic', 'partition', 'offset', 'timestamp', 'timestampType', 'key', from_avro('fixedValue', currentValueSchema.value, fromAvroOptions).alias('parsedValue')) \
#       .write \
#       .format("delta") \
#       .mode("append") \
#       .option("mergeSchema", "true") \
#      .save(deltaTablePath)

In [0]:

def parse_and_save(df, epoch_id):
    fromAvroOptions = {"mode": "FAILFAST"}

    def getSchema(id):
        return str(schema_registry_client.get_schema(id).schema_str)

    cached_df = df.cache()
    distinct_schema_ids = cached_df.select(col("valueSchemaId").cast("integer")).distinct()

    for row in distinct_schema_ids.collect():
        current_schema_id = row.valueSchemaId
        schema_str = getSchema(current_schema_id)

        # Filter for the schema ID
        filter_df = cached_df.filter(col("valueSchemaId") == current_schema_id)

        # Parse Avro and flatten fields
        parsed_df = filter_df.select(
            col("topic"), col("partition"), col("offset"), col("timestamp"), col("timestampType"), col("key"),
            from_avro("fixedValue", schema_str, fromAvroOptions).alias("parsedValue")
        )

        flat_df = parsed_df.select(
            col("topic"), col("partition"), col("offset"), col("timestamp"), col("timestampType"), col("key"),
            col('parsedValue.Id').alias('patient_id'),
            col('parsedValue.BIRTHDATE').alias('date_of_birth'),
            col('parsedValue.DEATHDATE').alias('date_of_death'),
            col('parsedValue.SSN').alias('ssn'),
            col('parsedValue.DRIVERS').alias('drivers'),
            col('parsedValue.PASSPORT').alias('passport'),
            col('parsedValue.PREFIX').alias('prefix'),
            col('parsedValue.FIRST').alias('first'),
            col('parsedValue.MIDDLE').alias('middle'),
            col('parsedValue.LAST').alias('last'),
            col('parsedValue.SUFFIX').alias('suffix'),
            col('parsedValue.MAIDEN').alias('maiden'),
            col('parsedValue.MARITAL').alias('marital'),
            col('parsedValue.RACE').alias('race'),
            col('parsedValue.ETHNICITY').alias('ethnicity'),
            col('parsedValue.GENDER').alias('gender'),
            col('parsedValue.BIRTHPLACE').alias('birthplace'),
            col('parsedValue.ADDRESS').alias('address'),
            col('parsedValue.CITY').alias('city'),
            col('parsedValue.STATE').alias('state'),
            col('parsedValue.COUNTY').alias('county'),
            col('parsedValue.FIPS').alias('fips'),
            col('parsedValue.ZIP').alias('zip'),
            col('parsedValue.LAT').alias('lat'),
            col('parsedValue.LON').alias('lon'),
            col('parsedValue.HEALTHCARE_EXPENSES').alias('healthcare_expenses'),
            col('parsedValue.HEALTHCARE_COVERAGE').alias('healthcare_coverage'),
            col('parsedValue.INCOME').alias('income')
        )

        # Write to Unity Catalog table (replace this with your actual catalog.schema.table)
        flat_df.write.format("delta") \
            .mode("append") \
            .option("mergeSchema", "true") \
            .saveAsTable("healthcare.bronze_layer.patients")

In [0]:
streamTestDf.writeStream \
  .option("checkpointLocation", checkpointPath) \
  .foreachBatch(parse_and_save) \
  .queryName("clickStreamTestFromConfluent") \
  .start()

In [0]:
deltaTestDf = spark.read.format("delta").load(deltaTablePath)
display(deltaTestDf)


topic,partition,offset,timestamp,timestampType,key,parsedValue
patients,0,40,2025-04-14T17:27:53.426Z,0,patients,"List(bfc117e7-391c-1da2-ccfe-72fa090e76f7, 2009-09-29, , 999-87-7162, , , , Hae300, Guillermina633, Johnston597, , , , native, Unknown, F, Anchorage Alaska US, 694 Mann Loaf, Ketchikan, Alaska, Ketchikan Gateway Borough, 2130, 99901, 55.34109371993423, -131.64778185969251, 22366.73, 43543.8, 47715)"
patients,0,41,2025-04-14T17:27:53.426Z,0,patients,"List(e1f16389-4489-0191-67b2-bba278af3a1a, 2024-12-27, , 999-91-1081, , , , Nestor901, Buster609, Reilly981, , , , native, Unknown, M, Anchorage Alaska US, 353 Cassin Street Unit 27, Wainwright, Alaska, North Slope Borough, 2185, 99782, 70.65297714280867, -159.94925062830728, 1612.82, 0.0, 52253)"
patients,0,42,2025-04-14T17:27:53.426Z,0,patients,"List(f05f1ed7-1952-81fe-66e0-1d2616e4e98a, 2008-01-29, , 999-93-7212, S99944650, , , Wilbur107, Luigi346, Rohan584, , , , hawaiian, Unknown, M, Farmers Loop Alaska US, 625 Morar Gardens, Valdez, Alaska, Valdez-Cordova Census Area, 2261, 99686, 61.12281233053141, -146.8451423204503, 40235.45, 29561.33, 41960)"
patients,0,43,2025-04-14T17:27:53.426Z,0,patients,"List(1606e56d-c5f5-b8df-6b11-21da8dbf3cc6, 1988-12-28, , 999-20-2510, S99986111, X36330822X, Mr., Josef103, Casey401, Senger904, , , M, hawaiian, Unknown, M, Haines Alaska US, 301 Dickens Track, Anchorage, Alaska, Anchorage Municipality, 2020, 99515, 61.227322521773, -149.87134319080425, 167905.04, 45577.38, 72097)"
patients,0,44,2025-04-14T17:27:53.426Z,0,patients,"List(4e681547-043c-2dad-07d9-3bbb419e2816, 1997-02-19, , 999-91-3888, S99941223, X14437175X, Mrs., Tasia358, Jayne73, D'Amore443, , Feest103, M, native, Unknown, F, Anchorage Alaska US, 1075 McKenzie Junction, Ketchikan, Alaska, Ketchikan Gateway Borough, 2130, 99901, 55.550931333302415, -131.14433526819747, 40879.22, 1038154.1, 104095)"
patients,0,45,2025-04-14T17:27:53.426Z,0,patients,"List(037fe238-4f18-b705-bfe5-2c4933eac288, 1965-06-20, , 999-30-9001, S99928359, X78231430X, Mr., Yong583, Hung902, Keebler762, , , D, white, Unknown, M, College Alaska US, 603 Haley Mill, Nikiski, Alaska, Kenai Peninsula Borough, 2122, 99635, 60.73096200543891, -151.1939969721525, 201889.66, 0.0, 40264)"
patients,0,46,2025-04-14T17:27:53.427Z,0,patients,"List(efd6b3e7-8eaa-c97c-2cf0-6010fcb1dbba, 1983-03-19, , 999-12-1847, S99990604, X29280085X, Mr., Dorsey40, , Bernhard322, , , M, hawaiian, Unknown, M, Fairbanks Alaska US, 384 Kuphal Vista Apt 26, Anchorage, Alaska, Anchorage Municipality, 2020, 99506, 61.01814438093765, -149.38859710313392, 379982.27, 331965.29, 37964)"
patients,0,47,2025-04-14T17:27:53.427Z,0,patients,"List(f9ebed13-dd01-f343-6625-3111f08f655b, 2015-06-29, , 999-32-3123, , , , Jame231, Weldon459, Blick895, , , , white, Unknown, M, Sutton-Alpine Alaska US, 841 Dicki Ville, Fairbanks, Alaska, Fairbanks North Star Borough, 2090, 99705, 65.07953536778304, -145.9581209666679, 20585.96, 20393.09, 48159)"
patients,0,48,2025-04-14T17:27:53.427Z,0,patients,"List(fde1280f-c7d7-5b78-265e-7019bf6543dc, 1964-07-21, 2024-12-09, 999-44-3779, S99943067, X8886559X, Mr., Denny560, Tyrell880, Cremin516, , , M, white, Unknown, M, Fairbanks Alaska US, 319 Hickle Light Apt 76, Denali Park, Alaska, Denali Borough, 2068, 99755, 63.784246912594874, -148.4897222079668, 44916.22, 800052.26, 122016)"
patients,0,49,2025-04-14T17:27:53.427Z,0,patients,"List(67e693d3-47eb-eb5d-aa77-c7f40bf8a661, 1955-10-19, 2023-06-17, 999-28-3944, S99995094, X40553465X, Mrs., Ashley34, Jordan900, Morar593, , Nitzsche158, W, other, Unknown, F, Fairbanks Alaska US, 168 Will View Apt 42, Anchorage, Alaska, Anchorage Municipality, 2020, 99504, 61.22209155235967, -149.7106358719642, 1246157.3, 382702.51, 108938)"


In [0]:
# deltaTestDf.count()


24

In [0]:
# deltaTestDf.printSchema()

In [0]:

# # Assuming your DataFrame is named 'df'
# new_df = deltaTestDf.select(
#     col('topic'),
#     col('partition'),
#     col('offset'),
#     col('timestamp'),
#     col('timestampType'),
#     col('key'),
#     col('parsedValue.Id').alias('patient_id'),
#     col('parsedValue.BIRTHDATE').alias('date_of_birth'),
#     col('parsedValue.DEATHDATE').alias('date_of_death'),
#     col('parsedValue.SSN').alias('ssn'),
#     col('parsedValue.DRIVERS').alias('drivers'),
#     col('parsedValue.PASSPORT').alias('passport'),
#     col('parsedValue.PREFIX').alias('prefix'),
#     col('parsedValue.FIRST').alias('first'),
#     col('parsedValue.MIDDLE').alias('middle'),
#     col('parsedValue.LAST').alias('last'),
#     col('parsedValue.SUFFIX').alias('suffix'),
#     col('parsedValue.MAIDEN').alias('maiden'),
#     col('parsedValue.MARITAL').alias('marital'),
#     col('parsedValue.RACE').alias('race'),
#     col('parsedValue.ETHNICITY').alias('ethnicity'),
#     col('parsedValue.GENDER').alias('gender'),
#     col('parsedValue.BIRTHPLACE').alias('birthplace'),
#     col('parsedValue.ADDRESS').alias('address'),
#     col('parsedValue.CITY').alias('city'),
#     col('parsedValue.STATE').alias('state'),
#     col('parsedValue.COUNTY').alias('county'),
#     col('parsedValue.FIPS').alias('fips'),
#     col('parsedValue.ZIP').alias('zip'),
#     col('parsedValue.LAT').alias('lat'),
#     col('parsedValue.LON').alias('lon'),
#     col('parsedValue.HEALTHCARE_EXPENSES').alias('healthcare_expenses'),
#     col('parsedValue.HEALTHCARE_COVERAGE').alias('healthcare_coverage'),
#     col('parsedValue.INCOME').alias('income') 
# )

# # Now 'new_df' contains exploded columns from 'parsedValue'
# new_df.display()